In [ ]:
from lib.ekyn import get_ekyn_ids,load_ekyn_pt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,ConcatDataset
import math
from torch import nn
from lib.models import RegNet
from torch.nn.functional import relu
import matplotlib.pyplot as plt
import torch
from lib.datasets import EpochedDataset
from tqdm import tqdm

CONFIG = {
    'WINDOW_SIZE':5000,
    'BATCH_SIZE':512,
    'LEARNING_RATE':1e-2,
    'DEVICE':'cuda',
    'FOLD':8
}

train_idx = get_ekyn_ids()
test_idx = [train_idx[CONFIG['FOLD']]]
train_idx.remove(test_idx[0])
print(train_idx,test_idx)
print(len(train_idx),len(test_idx))

In [ ]:
trainloader = DataLoader(ConcatDataset([EpochedDataset(idx=idx,condition=condition) for idx in train_idx for condition in ['Vehicle','PF']]),batch_size=512,shuffle=True)
devloader = DataLoader(ConcatDataset([EpochedDataset(idx=idx,condition=condition) for idx in test_idx for condition in ['Vehicle','PF']]),batch_size=512,shuffle=True)

In [ ]:
model = RegNet(in_features=5000,in_channels=1,depthi=[2],widthi=[32])
# criterion = nn.CrossEntropyLoss(weight=torch.tensor([18.3846,  2.2810,  1.9716]))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),lr=CONFIG['LEARNING_RATE'],weight_decay=1e-2)

In [ ]:
model.to(CONFIG['DEVICE']);
criterion.to(CONFIG['DEVICE']);

In [ ]:
lossi = []
devlossi = []

In [ ]:
from lib.utils import development_loop
for i in range(5):
    for j,(Xi,yi) in enumerate(tqdm(trainloader)):
        Xi,yi = Xi.to(CONFIG['DEVICE']),yi.to(CONFIG['DEVICE'])
        logits = model(Xi)
        loss = criterion(logits,yi)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lossi.append(loss.item())
    loss,f1 = development_loop(model,devloader,criterion,CONFIG['DEVICE'])
    devlossi.append(loss)
loss

In [ ]:
plt.plot(lossi)
plt.plot(range(34,len(lossi) + 1,len(trainloader)),devlossi)
# plt.xlim([len(lossi),len(lossi)])
plt.yscale('log')
# plt.ylim([0,1])
torch.tensor(devlossi).min()

In [ ]:
from lib.utils import evaluate
from sklearn.metrics import ConfusionMatrixDisplay,classification_report

loss,report,y_true,y_pred,y_logits = evaluate(dataloader=trainloader,model=model,criterion=criterion,DEVICE=CONFIG['DEVICE'])
ConfusionMatrixDisplay.from_predictions(y_true,y_pred,normalize='true')
print(classification_report(y_true,y_pred))
print(loss)
loss,report,y_true,y_pred,y_logits = evaluate(dataloader=devloader,model=model,criterion=criterion,DEVICE=CONFIG['DEVICE'])
ConfusionMatrixDisplay.from_predictions(y_true,y_pred,normalize='true')
print(classification_report(y_true,y_pred))
print(loss)
# unweighted dev record
# tensor(0.2306)

# weighted dev record
# tensor(0.8262)

In [ ]:
import seaborn as sns
model.to('cpu')
Xi,yi = next(iter(trainloader))
feature_map_idx = 3
sns.kdeplot(model(Xi[torch.where(yi.argmax(axis=1) == 0)[0]],return_encoding=True)[:,feature_map_idx].detach().flatten());
sns.kdeplot(model(Xi[torch.where(yi.argmax(axis=1) == 1)[0]],return_encoding=True)[:,feature_map_idx].detach().flatten());
sns.kdeplot(model(Xi[torch.where(yi.argmax(axis=1) == 2)[0]],return_encoding=True)[:,feature_map_idx].detach().flatten());

In [ ]:
model.to('cpu')
Xi,yi = next(iter(trainloader))

fig,ax = plt.subplots(nrows=len(model.c1.weight),ncols=2,figsize=(8,10))
for i,kernel in enumerate(model.c1.weight.squeeze().detach()):
    ax[i,0].plot(kernel)
for i,kernel in enumerate(model.c1(Xi.reshape(-1,1,5000)).detach()[0]):
    ax[i,1].plot(kernel)